# Setup Cloud SQL - SQL Alchemy
---

Install [pymysql](https://docs.sqlalchemy.org/en/14/dialects/mysql.html#module-sqlalchemy.dialects.mysql.pymysql)

In [14]:
!pip install pymysql

     |████████████████████████████████| 43 kB 1.4 MB/s 


Authenticate using ***application default credentials***

In [ ]:
!gcloud auth application-default login

Need to add new network if use public IP: `0.0.0.0/0`



### Code

In [11]:
# import create_engine from sql alchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

# import pandas
import pandas as pd

# import drive
from google.colab import drive

Mount gdrive folder to get SSL files

In [12]:
drive.mount('/content/drive')

Mounted at /content/drive


Build connection without SSL

In [ ]:
engine = create_engine(
  URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="CKnEx4mOtx2EADil",
    host="34.72.191.193",
    port=3306,
    database="mysql"
  )
)

conn = engine.connect()

Build connection with SSL

In [15]:
path_to_ssl = '/content/drive/MyDrive/Colab Notebooks/creds/SSL'

ssl_files={
    "ssl_ca": f"{path_to_ssl}/server-ca.pem",
    "ssl_cert": f"{path_to_ssl}/client-cert.pem",
    "ssl_key": f"{path_to_ssl}/client-key.pem"
}

engine = create_engine(
  URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="CKnEx4mOtx2EADil",
    host="34.72.191.193",
    port=3306,
    database="dummy_db"
  ),
  connect_args=ssl_files
)

conn = engine.connect()

# Create Table & Insert Data
---

To create table *(this query doesn't return a result)*

In [23]:
query = """
CREATE TABLE IF NOT EXISTS post (
  id INT(3) PRIMARY KEY AUTO_INCREMENT,
  company_name VARCHAR(255),
  skills VARCHAR(255),
  more_info VARCHAR(255),
  publish_date VARCHAR(255)
)
"""

query_result = conn.execute(query)

# Web Scraping Using Beautiful Soup
---

Mount gdrive folder

In [24]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Change directory to the working directory

In [25]:
%cd /content/drive/MyDrive/Colab Notebooks/github/web-scraping-to-cloud-sql

/content/drive/MyDrive/Colab Notebooks/github/web-scraping-to-cloud-sql


Import Libraries

In [26]:
# import beautiful soup
from bs4 import BeautifulSoup

import requests
# import requests

# import time
import time

Define `fetch_html` function that fetch HTML text from given URL per pages

In [27]:
def fetch_html(page):
  base_url = 'https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=python'
  page_param = f'&sequence={page}'
  html_content = requests.get(f'{base_url}{page_param}')
  html_text = html_content.text
  
  return html_text

Define `parse_html` function that parse HTML text using [beautiful soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) & html parser

In [28]:
def parse_html(html_text):
  soup = BeautifulSoup(html_text, 'html.parser')

  return soup

Get user input

In [29]:
print('Input how many pages you want to scrape?')
page_number = input('> ')

Input how many pages you want to scrape?
> 3


Define `find_jobs()` function that scraping **company**, **skill**, & **publish date** info

In [30]:
def find_jobs(soup):
  jobs = soup.find_all('li', class_ = 'clearfix job-bx wht-shd-bx')

  list_of_dict_jobs = []

  for idx, job in enumerate(jobs):
    company = job.find('h3', class_ = 'joblist-comp-name').text.replace(' ', ' ').strip()
    skills = job.find('span', class_ = 'srp-skills').text.strip()
    publish_date = job.find('span', class_ = 'sim-posted').text.strip()
    more_info = job.header.h2.a['href']

    dict_job = {}
    dict_job['company'] = company
    dict_job['skills'] = skills
    dict_job['publish_date'] = publish_date
    dict_job['more_info'] = more_info

    list_of_dict_jobs.append(dict_job)

  return list_of_dict_jobs

Define `write_to_file` function that writes jobs into `.txt` file for every page

In [31]:
def write_to_file(jobs, page):
  with open(f"posts/{page}.txt", "a") as f:
    for job in jobs:
      f.write(f"company name: {job.get('company')} \n")
      f.write(f"skills: {job.get('skills')} \n")
      f.write(f"more info: {job.get('more_info')} \n")
      f.write(f"publish date: {job.get('publish_date')} \n")
      f.write(f"--------------------------------------- \n")
    
  print(f"file saved: {page}")

Main Code

In [35]:
result_jobs = []

for idx in range(int(page_number)):
  html_text = fetch_html(idx+1)
  soup = parse_html(html_text)
  jobs = find_jobs(soup)
  write_to_file(jobs, idx+1)

  result_jobs += jobs

file saved: 1
file saved: 2
file saved: 3


# Insert data to db
---

In [37]:
query="""
  INSERT INTO dummy_db.post (`company_name` ,`skills` ,`more_info` ,`publish_date`) 
  VALUES (%s,%s,%s,%s)
"""

num_of_rows_added = 0
for job in result_jobs:
  data = (job['company'], job['skills'], job['more_info'], job['publish_date'])
  id=conn.execute(query, data)
  num_of_rows_added += id.rowcount

print(f'success inserted {num_of_rows_added} rows')

success inserted 75 rows


To show a table

Fetch methods:


1.   `.fetchall()`: to return all rows of query result
2.   `.fetchmany(size)`: to return number of rows specified by **size**
3.   `.fetchone()`: to return only the 1st row of query result



In [38]:
query_get_cols = "SHOW COLUMNS FROM dummy_db.post"
query_result = conn.execute(query_get_cols)
column_names = query_result.fetchall()
columns = [col[0] for col in column_names]

query_get_data = "select * from dummy_db.post"
query_result = conn.execute(query_get_data)
data = query_result.fetchall()

df = pd.DataFrame(data, columns=columns)
df

,id,company_name,skills,more_info,publish_date
0,1,Pure Tech Codex Private Limited,"rest , python , database , django , de...",https://www.timesjobs.com/job-detail/python-pu...,Posted 1 day ago
1,2,Surya Informatics Solutions Pvt. Ltd.,"python , web technologies , linux , mobi...",https://www.timesjobs.com/job-detail/python-su...,Posted few days ago
2,3,Gemini Solutions,"python , mobile , svn , nosql , python...",https://www.timesjobs.com/job-detail/qa-python...,Posted few days ago
3,4,Electrobrain modern technologies pvt ltd\r\n ...,"CSS , HTML , Java , Sql , c ++ , Oracle",https://www.timesjobs.com/job-detail/python-de...,Posted a month ago
4,5,GT SOLUSN\r\n (More Jobs),"""Python""",https://www.timesjobs.com/job-detail/python-de...,Posted few days ago
...,...,...,...,...,...
70,71,METADESIGN SOLUTIONS,"python , django , html5 , javascript",https://www.timesjobs.com/job-detail/python-de...,Posted few days ago
71,72,Zealogics LLC.,"san , python , css , javascript , jque...",https://www.timesjobs.com/job-detail/python-de...,Posted few days ago
72,73,Merkle Inc.,"softwaredesign , devops , python , nosql...",https://www.timesjobs.com/job-detail/python-de...,Posted 6 days ago
73,74,zf friedrichshafen -itc,"fundamentals , algorithms , c , softwa...",https://www.timesjobs.com/job-detail/python-fu...,Posted 6 days ago


To truncate table *(this query doesn't return a result)*

In [21]:
query = "TRUNCATE TABLE dummy_db.post"

query_result = conn.execute(query)